In [52]:
#Import all necessary libraries
import pandas as pd 
import numpy as np 
import re
import nltk
import spacy
import string
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

###IMPORT AND VIEW THE DATASET###

In [7]:
#Import the dataset
review=pd.read_csv('/Users/khai/Desktop/Sentiment Analysis/Restaurant_Reviews.tsv',sep='\t',quoting=3)
print(review.head())
print(review.shape)

                                              Review  Liked
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1
(1000, 2)


In [88]:
#Plot number of liked and no liked
review.head(7)

,Review,Liked,lowertext,nopuntext,stopwtext,lemmtext
0,Wow... Loved this place.,1,wow... loved this place.,wow loved this place,wow loved place,wow loved place
1,Crust is not good.,0,crust is not good.,crust is not good,crust not good,crust not good
2,Not tasty and the texture was just nasty.,0,not tasty and the texture was just nasty.,not tasty and the texture was just nasty,not tasty texture nasty,not tasty texture nasty
3,Stopped by during the late May bank holiday of...,1,stopped by during the late may bank holiday of...,stopped by during the late may bank holiday of...,stopped late may bank holiday rick steve recom...,stopped late may bank holiday rick steve recom...
4,The selection on the menu was great and so wer...,1,the selection on the menu was great and so wer...,the selection on the menu was great and so wer...,selection menu great prices,selection menu great price
5,Now I am getting angry and I want my damn pho.,0,now i am getting angry and i want my damn pho.,now i am getting angry and i want my damn pho,getting angry want damn pho,getting angry want damn pho
6,Honeslty it didn't taste THAT fresh.),0,honeslty it didn't taste that fresh.),honeslty it didnt taste that fresh,honeslty didnt taste fresh,honeslty didnt taste fresh


TEXT CLEANING

In [8]:
#Lower case the letters
review["lowertext"] = review["Review"].str.lower() 
review["lowertext"].head()

0                             wow... loved this place.
1                                   crust is not good.
2            not tasty and the texture was just nasty.
3    stopped by during the late may bank holiday of...
4    the selection on the menu was great and so wer...
Name: lowertext, dtype: object

In [9]:
#Remove punctuation
PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','', PUNCT))
review["nopuntext"] = review["lowertext"].apply(lambda text: remove_punctuation(text))
review["nopuntext"].head()

0                                 wow loved this place
1                                    crust is not good
2             not tasty and the texture was just nasty
3    stopped by during the late may bank holiday of...
4    the selection on the menu was great and so wer...
Name: nopuntext, dtype: object

In [10]:
#Remove STOPWORDS
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPW = set(stopwords.words('english'))
to_delete = ["no","nor","aren't",'couldn',"couldn't",'didn',"didn't",'doesn',
            "doesn't",'don',"don't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',
             "isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'no','not', "shan't",
            'shan',"shan't",'shouldn',"shouldn't","that'll",'wasn',"wasn't",'weren',"weren't","won't",
            'wouldn',"wouldn't"]
STOPW.difference_update(to_delete)
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPW])
review["stopwtext"] = review["nopuntext"].apply(lambda text: remove_stopwords(text))
review["stopwtext"].head(15)

0                                       wow loved place
1                                        crust not good
2                               not tasty texture nasty
3     stopped late may bank holiday rick steve recom...
4                           selection menu great prices
5                           getting angry want damn pho
6                            honeslty didnt taste fresh
7     potatoes like rubber could tell made ahead tim...
8                                           fries great
9                                           great touch
10                                       service prompt
11                                    would not go back
12    cashier no care ever say still ended wayyy ove...
13          tried cape cod ravoli chicken cranberrymmmm
14                     disgusted pretty sure human hair
Name: stopwtext, dtype: object

In [11]:
#Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

[nltk_data] Downloading package wordnet to /Users/khai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
review["lemmtext"] = review["stopwtext"].apply(lambda text:lemmatize_words(text))
review["lemmtext"].head(15)

0                                       wow loved place
1                                        crust not good
2                               not tasty texture nasty
3     stopped late may bank holiday rick steve recom...
4                            selection menu great price
5                           getting angry want damn pho
6                            honeslty didnt taste fresh
7     potato like rubber could tell made ahead time ...
8                                             fry great
9                                           great touch
10                                       service prompt
11                                    would not go back
12    cashier no care ever say still ended wayyy ove...
13          tried cape cod ravoli chicken cranberrymmmm
14                     disgusted pretty sure human hair
Name: lemmtext, dtype: object

In [13]:
#Retrieve the clean data
clean_review=review[['Review','lemmtext','Liked']]
clean_review.head()

,Review,lemmtext,Liked
0,Wow... Loved this place.,wow loved place,1
1,Crust is not good.,crust not good,0
2,Not tasty and the texture was just nasty.,not tasty texture nasty,0
3,Stopped by during the late May bank holiday of...,stopped late may bank holiday rick steve recom...,1
4,The selection on the menu was great and so wer...,selection menu great price,1


Adding new features

In [14]:
# Average word length feature
def avg_word_length(x):
    words=x.split()
    words_length=[len(word) for word in words]
    avg_word_length=sum(words_length)/len(words)
    return avg_word_length


In [15]:
clean_review['avg_word_len']=clean_review["Review"].apply(lambda text: avg_word_length(text))
clean_review.head(10)

<ipython-input-15-92d2eeb2644a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_review['avg_word_len']=clean_review["Review"].apply(lambda text: avg_word_length(text))


,Review,lemmtext,Liked,avg_word_len
0,Wow... Loved this place.,wow loved place,1,5.250000
1,Crust is not good.,crust not good,0,3.750000
2,Not tasty and the texture was just nasty.,not tasty texture nasty,0,4.250000
3,Stopped by during the late May bank holiday of...,stopped late may bank holiday rick steve recom...,1,4.866667
4,The selection on the menu was great and so wer...,selection menu great price,1,4.000000
5,Now I am getting angry and I want my damn pho.,getting angry want damn pho,0,3.272727
6,Honeslty it didn't taste THAT fresh.),honeslty didnt taste fresh,0,5.333333
7,The potatoes were like rubber and you could te...,potato like rubber could tell made ahead time ...,0,4.090909
8,The fries were great too.,fry great,1,4.200000
9,A great touch.,great touch,1,4.000000


In [16]:
#Number of words
def count_word(x):
    words=x.split()
    return len(words)
clean_review['sentence_length']=clean_review["Review"].apply(lambda text: count_word(text))
clean_review.head(10)

<ipython-input-16-fd8572471e95>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_review['sentence_length']=clean_review["Review"].apply(lambda text: count_word(text))


,Review,lemmtext,Liked,avg_word_len,sentence_length
0,Wow... Loved this place.,wow loved place,1,5.250000,4
1,Crust is not good.,crust not good,0,3.750000,4
2,Not tasty and the texture was just nasty.,not tasty texture nasty,0,4.250000,8
3,Stopped by during the late May bank holiday of...,stopped late may bank holiday rick steve recom...,1,4.866667,15
4,The selection on the menu was great and so wer...,selection menu great price,1,4.000000,12
5,Now I am getting angry and I want my damn pho.,getting angry want damn pho,0,3.272727,11
6,Honeslty it didn't taste THAT fresh.),honeslty didnt taste fresh,0,5.333333,6
7,The potatoes were like rubber and you could te...,potato like rubber could tell made ahead time ...,0,4.090909,22
8,The fries were great too.,fry great,1,4.200000,5
9,A great touch.,great touch,1,4.000000,3


In [17]:
#Special characters
def exclamationmark_count(x):
    words=x.split()
    exmark=[word for word in words if word.startswith('!')]
    exmark_count=[len(exmark[i].split('!'))-1 for i in range(0,len(exmark))]
    return sum(exmark_count)
clean_review['exclamationmark_count']=clean_review["Review"].apply(lambda text: exclamationmark_count(text))
clean_review.head(10)

,Review,lemmtext,Liked,avg_word_len,sentence_length,exclamationmark_count
0,Wow... Loved this place.,wow loved place,1,5.250000,4,0
1,Crust is not good.,crust not good,0,3.750000,4,0
2,Not tasty and the texture was just nasty.,not tasty texture nasty,0,4.250000,8,0
3,Stopped by during the late May bank holiday of...,stopped late may bank holiday rick steve recom...,1,4.866667,15,0
4,The selection on the menu was great and so wer...,selection menu great price,1,4.000000,12,0
5,Now I am getting angry and I want my damn pho.,getting angry want damn pho,0,3.272727,11,0
6,Honeslty it didn't taste THAT fresh.),honeslty didnt taste fresh,0,5.333333,6,0
7,The potatoes were like rubber and you could te...,potato like rubber could tell made ahead time ...,0,4.090909,22,0
8,The fries were great too.,fry great,1,4.200000,5,0
9,A great touch.,great touch,1,4.000000,3,0


# NGrams model prepare for machine learning model

In [18]:
from sklearn.model_selection import train_test_split
#Split the train test data
X=clean_review.drop(['Liked','Review'],axis=1)
y=clean_review[['Liked']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42,stratify=y)


In [19]:
#Create ngram modelling
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(ngram_range=(1,2),max_features=1000)
X_train_txt=vect.fit_transform(X_train['lemmtext'])
X_train_txt=pd.DataFrame(X_train_txt.toarray(),columns=vect.get_feature_names())
X_train_txt

,10,10 minute,12,20,20 minute,30,30 min,30 minute,40,40 minute,...,wow,wrong,year,year ago,yet,youd,youre,yummy,zero,zero star
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_test_txt=vect.transform(X_test['lemmtext'])
X_test_txt=pd.DataFrame(X_test_txt.toarray(),columns=vect.get_feature_names())
X_test_txt.head()

,10,10 minute,12,20,20 minute,30,30 min,30 minute,40,40 minute,...,wow,wrong,year,year ago,yet,youd,youre,yummy,zero,zero star
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#Combine with other features
X_train.index=range(0,700)
X_train_final=pd.concat([X_train_txt,X_train[['avg_word_len','sentence_length','exclamationmark_count']]],axis=1)
X_train_final.head()

,10,10 minute,12,20,20 minute,30,30 min,30 minute,40,40 minute,...,year ago,yet,youd,youre,yummy,zero,zero star,avg_word_len,sentence_length,exclamationmark_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.545455,11,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.272727,11,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.500000,8,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.666667,3,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.444444,9,0


In [22]:
X_test.index=range(0,300)
X_test_final=pd.concat([X_test_txt,X_test[['avg_word_len','sentence_length','exclamationmark_count']]],axis=1)
X_test_final.head()

,10,10 minute,12,20,20 minute,30,30 min,30 minute,40,40 minute,...,year ago,yet,youd,youre,yummy,zero,zero star,avg_word_len,sentence_length,exclamationmark_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.947368,19,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.300000,10,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.850000,20,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.166667,12,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.888889,9,0


In [23]:
X_train_final=X_train_final.values
X_test_final=X_test_final.values
y_train=y_train.values
y_test=y_test.values

# Fitting Random Forest

In [24]:
from sklearn.metrics import accuracy_score,make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier as rbf

In [58]:
pipe_rf = make_pipeline(rbf())

In [59]:
rf_param_grid = {

    'randomforestclassifier__criterion':['gini','entropy'],
    'randomforestclassifier__max_features': ['auto','sqrt','log2',None],
    'randomforestclassifier__max_depth': np.arange(1,50,1),
    'randomforestclassifier__n_estimators': np.arange(1,200,5),
    'randomforestclassifier__min_weight_fraction_leaf': np.arange(0,1,0.05),
    'randomforestclassifier__min_samples_split':np.arange(1,50,1)
    
}

# Perform RandomizedSearchCV
randomized_search_rf = RandomizedSearchCV(estimator=pipe_rf,param_distributions=rf_param_grid,n_iter=100,scoring=make_scorer(accuracy_score),verbose=1,cv=4)

In [60]:
randomized_search_rf.fit(X_train_final,y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estim

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/joblib/paral

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y w

...

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/joblib/paral

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y w

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/joblib/paral

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:   41.9s finished
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('randomforestclassifier',
                                              RandomForestClassifier())]),
                   n_iter=100,
                   param_distributions={'randomforestclassifier__criterion': ['gini',
                                                                              'entropy'],
                                        'randomforestclassifier__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42...
                                        'randomforestclassifier__min_weight_fraction_leaf': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]),
                                        'randomforestclassifier__n_estimators': array([  1,   6,  11,  16,  21,  26,  31,  36

In [61]:
print(randomized_search_rf.best_params_)
print(randomized_search_rf.best_score_)

{'randomforestclassifier__n_estimators': 166, 'randomforestclassifier__min_weight_fraction_leaf': 0.0, 'randomforestclassifier__min_samples_split': 8, 'randomforestclassifier__max_features': 'sqrt', 'randomforestclassifier__max_depth': 43, 'randomforestclassifier__criterion': 'gini'}
0.7785714285714286


In [62]:
rf_pred=randomized_search_rf.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, rf_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,rf_pred)
score2 = precision_score(y_test,rf_pred)
score3= recall_score(y_test,rf_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[122  28]
 [ 40 110]]


Accuracy is  77.33 %
Precision is  0.8
Recall is  0.73


# Fitting Logistic Regression

In [63]:
from sklearn.linear_model import LogisticRegression

In [64]:
pipe_logis=make_pipeline(LogisticRegression())
pipe_logis

Pipeline(steps=[('logisticregression', LogisticRegression())])

In [65]:
logis_param_grid = {
    'logisticregression__l1_ratio':np.arange(0,1,0.05),
    'logisticregression__C':np.arange(0,10,0.5)}

# Perform RandomizedSearchCV
randomized_search_logis = RandomizedSearchCV(estimator=pipe_logis,param_distributions=logis_param_grid,n_iter=100,scoring=make_scorer(accuracy_score),verbose=1,cv=4)

In [66]:
randomized_search_logis.fit(X_train_final,y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver option

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

...

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exa

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('logisticregression',
                                              LogisticRegression())]),
                   n_iter=100,
                   param_distributions={'logisticregression__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5]),
                                        'logisticregression__l1_ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])},
                   scoring=make_scorer(accuracy_score), verbose=1)

In [67]:
print(randomized_search_logis.best_params_)
print(randomized_search_logis.best_score_)

{'logisticregression__l1_ratio': 0.15000000000000002, 'logisticregression__C': 5.5}
0.79


In [68]:
logis_pred=randomized_search_logis.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, logis_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,logis_pred)
score2 = precision_score(y_test,logis_pred)
score3= recall_score(y_test,logis_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[121  29]
 [ 31 119]]


Accuracy is  80.0 %
Precision is  0.8
Recall is  0.79


# Naive Bayes

In [71]:
from sklearn.naive_bayes import GaussianNB

In [74]:
gnb = GaussianNB()
gnb.fit(X_train_final,y_train)

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()

In [75]:
gnb_pred=gnb.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, gnb_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,gnb_pred)
score2 = precision_score(y_test,gnb_pred)
score3= recall_score(y_test,gnb_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[ 90  60]
 [ 14 136]]


Accuracy is  75.33 %
Precision is  0.69
Recall is  0.91


# Bernouli Bayes

In [76]:
from sklearn.naive_bayes import BernoulliNB
classifier_BB = BernoulliNB(alpha=0.8)
classifier_BB.fit(X_train_final,y_train)

# Predicting the Test set results
y_pred = classifier_BB.predict(X_test_final)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[109  41]
 [ 23 127]]


Accuracy is  78.67 %
Precision is  0.76
Recall is  0.85


/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


# Multinormal Bayes

In [77]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier_multiNB = MultinomialNB(alpha=0.1)
classifier_multiNB.fit(X_train_final, y_train)

# Predicting the Test set results
y_pred = classifier_multiNB.predict(X_test_final)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred)
score3= recall_score(y_test,y_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[115  35]
 [ 25 125]]


Accuracy is  80.0 %
Precision is  0.78
Recall is  0.83


/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


# Gradient Boosting

In [47]:
from sklearn.ensemble import GradientBoostingClassifier as gbr
pipe_gbr=make_pipeline(gbr())
pipe_gbr


Pipeline(steps=[('gradientboostingclassifier', GradientBoostingClassifier())])

In [48]:
gbr_param_grid = {
    'gradientboostingclassifier__learning_rate':np.arange(0.1,2,0.1),
    'gradientboostingclassifier__max_features': ['auto','sqrt','log2',None],
    'gradientboostingclassifier__max_depth': np.arange(1,50,1),
    'gradientboostingclassifier__n_estimators': np.arange(1,200,5),
    'gradientboostingclassifier__min_weight_fraction_leaf': np.arange(0,1,0.05),
    'gradientboostingclassifier__min_samples_split':np.arange(1,50,1)
    
}

# Perform RandomizedSearchCV
randomized_search_gbr = RandomizedSearchCV(estimator=pipe_gbr,param_distributions=gbr_param_grid,n_iter=100,scoring=make_scorer(accuracy_score),verbose=1,cv=4)

In [49]:
randomized_search_gbr.fit(X_train_final, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

...

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 498, in fit
    n_stages = self._fit_stages(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 555, in _fit_stages
    raw_predictions = self._fit_stage(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 211, in _fi

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 498, in fit
    n_stages = self._fit_s

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('gradientboostingclassifier',
                                              GradientBoostingClassifier())]),
                   n_iter=100,
                   param_distributions={'gradientboostingclassifier__learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                        'gradientboostingclassifier__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 1...
                                        'gradientboostingclassifier__min_weight_fraction_leaf': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]),
                                        'gradientboostingclassifier__n_estimators': array([  1,   6,  11,  16,  21,  26,  31,  36,  41,  46,  51,  56,  61,
        66,  71,  76,  81,  86,  91,  96, 101, 106, 111, 116

In [50]:
print(randomized_search_gbr.best_params_)
print(randomized_search_gbr.best_score_)




{'gradientboostingclassifier__n_estimators': 51, 'gradientboostingclassifier__min_weight_fraction_leaf': 0.0, 'gradientboostingclassifier__min_samples_split': 19, 'gradientboostingclassifier__max_features': 'auto', 'gradientboostingclassifier__max_depth': 31, 'gradientboostingclassifier__learning_rate': 1.2000000000000002}
0.7514285714285713


In [53]:
gbr_pred=randomized_search_gbr.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, gbr_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,gbr_pred)
score2 = precision_score(y_test,gbr_pred)
score3= recall_score(y_test,gbr_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[131  19]
 [ 46 104]]


Accuracy is  78.33 %
Precision is  0.85
Recall is  0.69


# Voting Classifer

In [80]:
from sklearn.ensemble import VotingClassifier
clf_voting = VotingClassifier(voting='soft',estimators=[('gbr', randomized_search_gbr),('randomforest',randomized_search_rf),
                                                        ('logistic',randomized_search_logis),('naive bayes',gnb),
                                                       ('bernouli bayes',classifier_BB),('multinormal bayes',classifier_multiNB)])
clf_voting.fit(X_train_final, y_train)

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 4 folds for each of 100 candidates, totalling 400 fits


/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 498, in fit
    n_stages = self._fit_stages(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 555, in _fit_stages
    raw_predictions = self._fit_stage(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 211, in _fi

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 498, in fit
    n_stages = self._fit_stages(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 555, in _fit_stages
    raw_predictions = self._fit_stage(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 211, in _fi

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py", line 335, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 498, in fit
    n_stages = self._fit_stages(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 555, in _fit_stages
    raw_predictions = self._fit_stage(
  File "/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py", line 211, in _fi

...

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of ite

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of ite

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of ite

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1315: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn("l1_ratio parameter is only used when penalty is "
/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of ite

VotingClassifier(estimators=[('gbr',
                              RandomizedSearchCV(cv=4,
                                                 estimator=Pipeline(steps=[('gradientboostingclassifier',
                                                                            GradientBoostingClassifier())]),
                                                 n_iter=100,
                                                 param_distributions={'gradientboostingclassifier__learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                                                      'gradientboostingclassifier__max_depth': ar...
                                                 param_distributions={'logisticregression__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5]),
                                                                      'logisti

In [81]:
voting_pred=clf_voting.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, voting_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,voting_pred)
score2 = precision_score(y_test,voting_pred)
score3= recall_score(y_test,voting_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[111  39]
 [ 18 132]]


Accuracy is  81.0 %
Precision is  0.77
Recall is  0.88


# Stacking

In [83]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier

In [84]:
reg_meta =LogisticRegression()

# Build the Stacking Classifier
reg_stack = StackingClassifier(estimators=[('gbr', randomized_search_gbr),('randomforest',randomized_search_rf),
                                           ('logistic',randomized_search_logis),('naive bayes',gnb),
                                        ('bernouli bayes',classifier_BB),('multinormal bayes',classifier_multiNB)],final_estimator=reg_meta,n_jobs=4)

#Fit in the model
reg_stack.fit(X_train_final,y_train)

/Users/khai/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


StackingClassifier(estimators=[('gbr',
                                RandomizedSearchCV(cv=4,
                                                   estimator=Pipeline(steps=[('gradientboostingclassifier',
                                                                              GradientBoostingClassifier())]),
                                                   n_iter=100,
                                                   param_distributions={'gradientboostingclassifier__learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                                                        'gradientboostingclassifier__max_depth':...
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5]),
                                                                        'logisticregression__l1_ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.

In [86]:
stacking_pred=reg_stack.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, stacking_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,stacking_pred)
score2 = precision_score(y_test,stacking_pred)
score3= recall_score(y_test,stacking_pred)
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[124  26]
 [ 33 117]]


Accuracy is  80.33 %
Precision is  0.82
Recall is  0.78
